In [3]:
import pandas as pd 
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


In [ ]:
data = pd.read_csv("file.csv")


In [ ]:
#DATA TRANSFORMATION PIPELINE 
#sequentially apply transformers then an estimator

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder


numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean'))])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')), ('encoder', OrdinalEncoder())])